In [1]:
import xarray as xr
import netCDF4 as nc
import numpy as np
from tqdm import tqdm

In [34]:
lat = 128 # chunk size 256,512
long = 128 # chunk size 256,512
def split_array(input_data, lat, long,region=100):
    """
    Split the whole 3600*1500 array into smaller chunks.
    Padding with zeros to make the array be divisible by chunk size.
    
    Args:
        input_data: data of a single day from a eta_t instance
        lat: chunk size
        long: chunk size
    """
    pad_cols = math.ceil(3600/long)*long - 3600 # number of zero columns adding to the right
    pad_rows = math.ceil(1500/lat)*lat - 1500 # number of zero rows adding to the bottom
    

    input_data=np.pad(input_data,((0,pad_rows),(0,pad_cols)), 'constant',constant_values=(0,0)) # padding with zeros, right & bottom
    
    l = np.array_split(input_data,len(input_data)/lat,axis=0)
    input_data_split = []
    for i in range(len(l)):
        dd = np.array_split(l[i],len(input_data[0])/long,axis=1)
        input_data_split += dd
    input_data_split = np.array(input_data_split)
    return input_data_split[region]

In [48]:
folder = 'dataset3_102' #change folder name for every region
num_of_years = 20
region = 102 #change region
num_errors = 0
error_year = []
error_month = []

basic_url = 'http://dapds00.nci.org.au/thredds/dodsC/gb6/BRAN/BRAN_2016/OFAM/ocean_eta_t_'
year = 1993
for i in range(num_of_years):
  year += 1
  month = 0
  for j in tqdm(range(12)):
    month += 1
    if month > 9:
      m = str(month)
    else:
      m = '0'+str(month)
    #print('year: ',year,' month: ',month)
    y = str(year)
    url = basic_url + y + '_' + m + '.nc'
    try:
        #data = nc.Dataset(url)
        data = xr.open_dataset(url)
    except:
        num_errors+=1
        error_year.append(y)
        error_month.append(m)
        continue
    #variables = data.variables
    eta_t = data['eta_t'].to_numpy() #data['eta_t'][:]
    size = eta_t.shape[0]
    for i in range(size):
      eta_t_day_i = eta_t[i]
      eta_t_numpy = eta_t_day_i #np.array(eta_t_day_i)
      inputs = split_array(eta_t_numpy,lat,long,region)
      inputsx = np.expand_dims(inputs,axis=2)
      file_name = './'+folder+'/128x128_'+y+'_'+m+'_'+str(i+1)+'.npy'
      np.save(file_name,inputsx) 

100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [18:19<00:00, 91.65s/it]
